In [44]:
from math import sqrt
import polars as pl
from plotly import express as px

In [45]:
audit = (
    # pl.read_ndjson("../generated/dev5_ACSEmployment_binarized.jsonl")
    pl.read_ndjson("../generated_srv/dev5_ACSEmployment_binarized_linear.jsonl")
    # .select(pl.all().exclude("model_params"))
    .group_by(
        "dataset",
        "model_name",
        "model_params",
        "strategy",
        "strategy_params",
        "audit_budget",
        "detection_tpr",
        "detection_tnr",
    )
    .mean()
    .with_columns(
        strategy_json=pl.col("strategy_params").struct.json_encode(),
        model_json=pl.col("model_params").struct.json_encode(),
    )
    .sort("detection_tpr", "detection_tnr", descending=[True, False])
)

In [46]:
fig = px.scatter(
    audit.filter(
        pl.col("detection_tnr") == 1.0,
        pl.col("detection_tpr") == 1.0,
        pl.col("strategy") == "honest",
    ),
    x="demographic_parity_user",
    y="utility_user",
    color="model_name",
    category_orders={"strategy": audit["strategy"].unique().sort().to_list()},
    height=600,
    width=800,
    title="Honest platform",
)
fig.show()

In [47]:
fig = px.scatter(
    audit.filter(
        pl.col("detection_tnr") == 1.0,
        pl.col("detection_tpr") == 0.5,
    ).with_columns(
        is_honest=pl.when(pl.col("strategy") == "honest")
        .then("strategy")
        .otherwise(pl.lit("sneaky")),
        size=10,
    ),
    x="demographic_parity_audit",
    y="utility_user",
    color="strategy",
    symbol="is_honest",
    size="size",
    size_max=10,
    hover_data=["strategy_json", "model_json", "model_name"],
    category_orders={"strategy": audit["strategy"].unique().sort().to_list()},
    height=600,
    width=800,
    title="Sneaky platform, detection tpr=50%",
)
fig.show()

In [48]:
fig = px.scatter(
    audit.with_columns(
        is_honest=pl.when(pl.col("strategy") == "honest")
        .then("strategy")
        .otherwise(pl.lit("sneaky")),
        size=10,
    ),
    x="demographic_parity_audit",
    y="utility_user",
    color="strategy",
    symbol="is_honest",
    size="size",
    size_max=10,
    facet_row="detection_tpr",
    facet_col="detection_tnr",
    hover_data=["strategy_json", "model_json", "model_name"],
    height=1_000,
    width=1_300,
)

# fig.update_xaxes(range=[0, 0.1])
# fig.update_yaxes(range=[0.81, 0.83])
fig.show()

# What is the best lying strategy ?

In [49]:
manipulation = (
    pl.read_ndjson("../generated/manipulation5_ACSEmployment_binarized_skrub.jsonl")
    .with_columns(
        min_conditional_accuracy=pl.col("conditional_accuracy_user").list.min(),
        strategy_instance=pl.concat_str(
            pl.col("strategy"), pl.col("strategy_params").struct.json_encode()
        ),
        strategy_json=pl.col("strategy_params").struct.json_encode(),
    )
    .group_by(
        "dataset",
        "model_name",
        # "model_params",
        "strategy",
        "strategy_params",
        "strategy_json",
        "audit_budget",
        "detection_tpr",
        "detection_tnr",
    )
    .agg(pl.col("*").mean(), pl.col("*").std().name.suffix("_std"))
    .sort("detection_tpr", "detection_tnr", descending=[True, False])
)

In [50]:
manipulation

dataset,model_name,strategy,strategy_params,strategy_json,audit_budget,detection_tpr,detection_tnr,model_params,entropy,fit_time,inference_time,utility_user,utility_audit,conditional_accuracy_user,demographic_parity_user,demographic_parity_audit,min_conditional_accuracy,strategy_instance,model_params_std,entropy_std,fit_time_std,inference_time_std,utility_user_std,utility_audit_std,conditional_accuracy_user_std,demographic_parity_user_std,demographic_parity_audit_std,min_conditional_accuracy_std,strategy_instance_std
str,str,str,struct[2],str,i64,f64,f64,struct[1],f64,f64,f64,f64,f64,list[f64],f64,f64,f64,str,struct[1],f64,f64,f64,f64,f64,list[f64],f64,f64,f64,str
"""ACSEmployment""","""unconstrained""","""randomized_response""","{3.0,null}","""{""epsilon"":3.0,""theta"":null}""",1000,1.0,1.0,null,2.3527e9,0.306453,0.050667,0.850877,0.853,null,0.07169,0.084962,0.829821,null,null,8.5239e8,0.0405,0.006901,0.002836,0.011392,null,0.008227,0.052824,0.025599,null
"""ACSEmployment""","""unconstrained""","""ROC_mitigation""","{null,0.566667}","""{""epsilon"":null,""theta"":0.5666…",1000,1.0,1.0,null,2.3527e9,0.338705,0.054933,0.850877,0.8509,null,0.07169,0.047842,0.833411,null,null,8.5239e8,0.073323,0.011531,0.002836,0.009457,null,0.008227,0.040339,0.021775,null
"""ACSEmployment""","""unconstrained""","""randomized_response""","{7.0,null}","""{""epsilon"":7.0,""theta"":null}""",1000,1.0,1.0,null,2.3527e9,0.322387,0.050837,0.850877,0.853,null,0.07169,0.084962,0.829821,null,null,8.5239e8,0.068567,0.009291,0.002836,0.011392,null,0.008227,0.052824,0.025599,null
"""ACSEmployment""","""unconstrained""","""ROC_mitigation""","{null,0.333333}","""{""epsilon"":null,""theta"":0.3333…",1000,1.0,1.0,null,2.3527e9,0.329613,0.055304,0.850877,0.853,null,0.07169,0.084962,0.829821,null,null,8.5239e8,0.059285,0.011079,0.002836,0.011392,null,0.008227,0.052824,0.025599,null
"""ACSEmployment""","""unconstrained""","""ROC_mitigation""","{null,0.433333}","""{""epsilon"":null,""theta"":0.4333…",1000,1.0,1.0,null,2.3527e9,0.344996,0.058376,0.850877,0.853,null,0.07169,0.084962,0.829821,null,null,8.5239e8,0.079822,0.016734,0.002836,0.011392,null,0.008227,0.052824,0.025599,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ACSEmployment""","""unconstrained""","""randomized_response""","{2.0,null}","""{""epsilon"":2.0,""theta"":null}""",1000,0.0,1.0,null,2.3527e9,0.318923,0.052718,0.850877,0.853,null,0.07169,0.084962,0.829821,null,null,8.5239e8,0.08329,0.012349,0.002836,0.011392,null,0.008227,0.052824,0.025599,null
"""ACSEmployment""","""unconstrained""","""ROC_mitigation""","{null,0.6}","""{""epsilon"":null,""theta"":0.6}""",1000,0.0,1.0,null,2.3527e9,0.308863,0.049406,0.850877,0.853,null,0.07169,0.084962,0.829821,null,null,8.5239e8,0.055975,0.007926,0.002836,0.011392,null,0.008227,0.052824,0.025599,null
"""ACSEmployment""","""unconstrained""","""honest""","{null,null}","""{""epsilon"":null,""theta"":null}""",1000,0.0,1.0,null,2.3527e9,0.327202,0.049787,0.850877,0.853,null,0.07169,0.084962,0.829821,null,null,8.5239e8,0.093703,0.0116,0.002836,0.011392,null,0.008227,0.052824,0.025599,null


In [51]:
px.line(
    manipulation.sort("detection_tpr").with_columns(
        pl.col("min_conditional_accuracy_std") / sqrt(5)
    ),
    x="detection_tpr",
    y="min_conditional_accuracy",
    error_y="min_conditional_accuracy_std",
    facet_col="strategy",
    color="strategy_json",
)

In [52]:
px.line(
    manipulation.sort("detection_tpr").with_columns(
        pl.col("demographic_parity_audit_std") / sqrt(5)
    ),
    x="detection_tpr",
    y="demographic_parity_audit",
    error_y="demographic_parity_audit_std",
    facet_col="strategy",
    color="strategy_json",
)

In [53]:
px.line(
    manipulation.sort("detection_tpr").with_columns(
        pl.col("demographic_parity_user_std") / sqrt(5)
    ),
    x="detection_tpr",
    y="demographic_parity_user",
    error_y="demographic_parity_user_std",
    facet_col="strategy",
    color="strategy_json",
)

In [54]:
px.line(
    manipulation.sort("detection_tpr").with_columns(
        pl.col("utility_user_std") / sqrt(5)
    ),
    x="detection_tpr",
    y="utility_user",
    error_y="utility_user_std",
    facet_col="strategy",
    color="strategy_json",
)

# Compute cost for the platform

In [55]:
px.bar(
    manipulation.group_by("strategy")
    .mean()
    .unpivot(
        index="strategy",
        on=["fit_time", "inference_time"],
        value_name="time",
        variable_name="phase",
    ),
    x="strategy",
    y="time",
    facet_col="phase",
    # color="strategy_json",
    # barmode="group",
)

# Finding datasets with the largest base rate difference

In [56]:
base_rate = pl.read_ndjson("../generated/disparities.jsonl")

In [57]:
px.line(
    base_rate,
    x="year",
    y="base_rate",
    color="state",
    facet_col="group",
    hover_data="len",
)